In [1]:
import pandas as pd
import requests
pd.set_option("
              
              
              
              
              
              
              
              
              
              
              
              
              
              
              play.max_rows", 15)-
#The energy data was uploaded to jupyter as a csv
data = pd.read_csv('system_energy_20131014_to_20170509_daily.csv')
data

,Date/Time,Energy Produced (Wh)
0,2013-10-14 00:00:00 -0700,2032
1,2013-10-15 00:00:00 -0700,29353
2,2013-10-16 00:00:00 -0700,28786
3,2013-10-17 00:00:00 -0700,29536
4,2013-10-18 00:00:00 -0700,29296
5,2013-10-19 00:00:00 -0700,29932
6,2013-10-20 00:00:00 -0700,28312
...,...,...
1298,2017-05-04 00:00:00 -0700,44857
1299,2017-05-05 00:00:00 -0700,30166


In [2]:
#drop the last row containing total
data.drop(len(data) -1, inplace= True)
data

,Date/Time,Energy Produced (Wh)
0,2013-10-14 00:00:00 -0700,2032
1,2013-10-15 00:00:00 -0700,29353
2,2013-10-16 00:00:00 -0700,28786
3,2013-10-17 00:00:00 -0700,29536
4,2013-10-18 00:00:00 -0700,29296
5,2013-10-19 00:00:00 -0700,29932
6,2013-10-20 00:00:00 -0700,28312
...,...,...
1297,2017-05-03 00:00:00 -0700,45582
1298,2017-05-04 00:00:00 -0700,44857


In [3]:
#get date stamps form energy csv to use in api calls.
data["Year"] = data['Date/Time'].str.slice(0,4)
data["Month"] = data['Date/Time'].str.slice(5,7)
data["Day"] = data['Date/Time'].str.slice(8,10)
data["Date"] = data.Year + data.Month + data.Day
data.to_csv("/home/tmgerrit/Data301/Final Project/Energy.csv", index=False)

In [4]:
#lat long of Santa Maria, used for getting sunrise and sunset times:SantaMaria latlong = [34.963623, -120.424576]
import sys
import time
SantaMarialoc = [34.963623, -120.424576]
sunrise = requests.get("https://api.sunrise-sunset.org/json?lat=34.963623&lng=-120.424576&date=20170607")
sr = "sunrise"
ss = "sunset"
dl = "day_length"

day_specs = {
    sr: [],
    ss: [],
    dl:[]
}

for day in data["Date"].values:
   
    sunrise = requests.get("https://api.sunrise-sunset.org/json?lat=34.963623&lng=-120.424576&date=%s&formatted=0"%day)
    res = sunrise.json()["results"]
    day_specs[sr].append(res[sr])
    day_specs[ss].append(res[ss])
    day_specs[dl].append(res[dl])
  
    sys.stdout.write('.')
    time.sleep(.001)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [5]:
pd.DataFrame(day_specs).to_csv("/home/tmgerrit/Data301/Final Project/Daytimes.csv", index=False)

In [6]:
weather = []
for year in range(2013, 2016):
    start = year
    end = year+1
    apicall = ("https://api.synopticlabs.org/v2/stations/timeseries?" +
               "&token=3066f4e105ff4614a61897b282e96cae&stid=KSMX&" +
               "start=%d10140000"%start+
               "&end=%d10140000"%end+
               "&timeformat=Y%25YM%25mD%25dH%25HM%25M"+
               "&vars=air_temp,weather_condition,relative_humidity,wind_speed,pressure")
    print(start, end)
    KSMXdata = requests.get(apicall)
    observationsjson = KSMXdata.json()['STATION'][0]["OBSERVATIONS"]
    weatherdf = pd.DataFrame(observationsjson)
    weather.append(weatherdf)
#get data from 10-14-2016 to current day
weather16_17 = requests.get('https://api.synopticlabs.org/v2/stations/timeseries?&token=3066f4e105ff4614a61897b282e96cae&stid=KSMX&start=201610140000&end=201705092000&timeformat=Y%25YM%25mD%25dH%25HM%25M&vars=air_temp,weather_condition,relative_humidity,wind_speed,pressure')
KSMXdata16_17 = weather16_17.json()['STATION'][0]["OBSERVATIONS"]
weather.append(pd.DataFrame(KSMXdata16_17))

2013 2014
2014 2015
2015 2016


In [7]:
weather_df = pd.concat(weather)
weather_df.to_csv("/home/tmgerrit/Data301/Final Project/Weather.csv", index=False)